In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
import json
import requests
import os
import geopandas as gpd
from pyproj import CRS
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,accuracy_score
import xgboost as xgb
from xgboost import plot_importance

In [ ]:
pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)

In [ ]:
train_df = pd.read_csv('./train_set.csv')
train_labels = pd.read_csv('./train_labels.csv')


In [ ]:
test_df = pd.read_csv('./test_set.csv')

In [ ]:
test_df.shape

In [ ]:
train_df.head()

In [ ]:
train_df = train_df.merge(train_labels,on='id')
train_df.head()

In [ ]:
train_df.shape

In [ ]:
train_df.info()

In [ ]:
for feature in train_df.columns:
    print('Feature:', feature, ' missing%:', (np.round(train_df[feature].isnull().mean(),4))*100)

In [ ]:
for feature in test_df.columns:
    print('Feature:', feature, ' missing%:', (np.round(test_df[feature].isnull().mean(),4))*100)

Handling missing values:

In [ ]:
train_df['funder'] = train_df['funder'].str.lower()
train_df['installer'] = train_df['installer'].str.lower()

In [ ]:
test_df['funder'] = test_df['funder'].str.lower()
test_df['installer'] = test_df['installer'].str.lower()

In [ ]:
train_df['funder'] = train_df['funder'].fillna('Unknown') 
train_df['installer'] = train_df['installer'].fillna('Unknown') 

In [ ]:
test_df['funder'] = test_df['funder'].fillna('Unknown') 
test_df['installer'] = test_df['installer'].fillna('Unknown') 

In [ ]:
train_df.loc[train_df['subvillage'].isnull(),'region'].value_counts()

In [ ]:
train_df.loc[train_df['subvillage'].isnull()]

Quite some information(gps_height,year of construction, amount_tsh,population) is not available or not know for the region of Dodoma. Removing data from this region having unknown subvillage.

Regions -> Districts(lga) -> Divisions -> Ward -> Subvillage

In [ ]:
train_df = train_df[train_df['subvillage'].notna()]

train_df['public_meeting'] = train_df['public_meeting'].fillna('Unknown') 
train_df['scheme_management'] = train_df['scheme_management'].fillna('Unknown') 

train_df = train_df.loc[:, train_df.columns != 'scheme_name']
train_df['permit'] = train_df['permit'].fillna('Unknown') 

In [ ]:
test_df = test_df[test_df['subvillage'].notna()]

test_df['public_meeting'] = test_df['public_meeting'].fillna('Unknown') 
test_df['scheme_management'] = test_df['scheme_management'].fillna('Unknown') 

test_df = test_df.loc[:, test_df.columns != 'scheme_name']
test_df['permit'] = test_df['permit'].fillna('Unknown') 

In [ ]:
train_df = train_df.reset_index(drop=True)

In [ ]:
test_df = test_df.reset_index(drop=True)

Total Static Head:

In [ ]:
train_df['amount_tsh'].describe()

In [ ]:
high_tsh = (len(train_df[train_df['amount_tsh'] > 9000])/len(train_df))*100
print('Only',np.round(high_tsh,4),'% waterpoints have greater than 9000 Total Static Head')

In [ ]:
train_df.loc[train_df['amount_tsh'] > 9000,'status_group'].value_counts()

Date Recorded:

In [ ]:
train_df['date_recorded'] = pd.to_datetime(train_df['date_recorded'])

In [ ]:
train_df['date_recorded'].describe()

In [ ]:
test_df['date_recorded'] = pd.to_datetime(test_df['date_recorded'])

In [ ]:
test_df.shape

Funder

In [ ]:
train_df['funder'].value_counts().head(10)

In [ ]:
len(train_df['funder'].unique())

GPS Height: 

In [ ]:
train_df['gps_height'].describe()

In [ ]:
train_df.loc[train_df['gps_height'] < 0.0,'waterpoint_type'].value_counts()

Negative GPS height here represents those waterpoints which have depth like a well, handpump etc. Height for these represents depth.

num_private:

In [ ]:
train_df['num_private'].describe()

In [ ]:
train_df = train_df.loc[:, train_df.columns != 'num_private']

In [ ]:
test_df = test_df.loc[:, test_df.columns != 'num_private']

Basins:

In [ ]:
train_df['basin'].value_counts()

In [ ]:
train_df['region'].value_counts()

Analysis:

Region, District and Ward wise all waterpoints:

In [ ]:
tanzania_regions_url = 'https://raw.githubusercontent.com/thadk/GeoTZ/master/TZA_adm1_mkoaTZ.geojson'
regions_json_data = json.loads(requests.get(tanzania_regions_url).text)
regions_df = gpd.GeoDataFrame.from_features(regions_json_data, crs='EPSG:4326')

In [ ]:
regions_df['geometry']

In [ ]:
grouped_by_region = train_df.groupby('region')
regionwise_waterpoints = pd.DataFrame()
regions = []
waterpoints = []
for name,group in grouped_by_region:
    if(name == 'Dar es Salaam'):
        name = 'Dar-Es-Salaam'
    regions.append(name)
    waterpoints.append(len(group))
regionwise_waterpoints['region'] = regions
regionwise_waterpoints['total_waterpoints'] = waterpoints

In [ ]:
merged = regions_df.merge(regionwise_waterpoints,left_on='NAME_1',right_on='region',how='left')

In [ ]:
regionwise_waterpoints = merged[['region','total_waterpoints','geometry']]

In [ ]:
regionwise_waterpoints = gpd.GeoDataFrame(
    regionwise_waterpoints, geometry='geometry')
regionwise_waterpoints.crs = CRS.from_epsg(4326)

In [ ]:
regionwise_waterpoints['geoid'] = regionwise_waterpoints.index.astype(str)

In [ ]:
m = folium.Map([6.3690,34.8888], zoom_start=4)

choropleth = folium.Choropleth(
    geo_data=regionwise_waterpoints,
    data=regionwise_waterpoints,
    columns=['geoid','total_waterpoints'],
    key_on='feature.id',
    nan_fill_color='purple',
    nan_fill_opacity=0.4,
    fill_color='YlGn',
    highlight=True
).add_to(m)

choropleth.geojson.add_child(folium.features.GeoJsonTooltip(
        fields=['region','total_waterpoints'],
        aliases=['Region','Waterpoints'],
        style=('background-color: grey; color: white;'),
        localize=True
        )
)

m

In [ ]:
district_url = 'https://raw.githubusercontent.com/thadk/GeoTZ/master/TZA_adm2_kiasi84pc%20detail.geojson'
districts_json_data = json.loads(requests.get(district_url).text)
districts_df = gpd.GeoDataFrame.from_features(districts_json_data, crs='EPSG:4326')

In [ ]:
grouped_by_districts = train_df.groupby(['region','lga'])

In [ ]:
districtswise_waterpoints = pd.DataFrame()
regions = []
districts = []
waterpoints = []
for name,group in grouped_by_districts:
    if(name[0] == 'Dar es Salaam'):
        regions.append('Dar-Es-Salaam')
    else:
        regions.append(name[0])
    districts.append(name[1])
    waterpoints.append(len(group))
districtswise_waterpoints['region'] = regions
districtswise_waterpoints['district'] = districts
districtswise_waterpoints['total_waterpoints'] = waterpoints

In [ ]:
merged = districts_df.merge(districtswise_waterpoints,left_on=['NAME_1','NAME_2'],right_on=['region','district'],how='left')

In [ ]:
districtswise_waterpoints = merged[['NAME_1','NAME_2','total_waterpoints','geometry']]

In [ ]:
districtswise_waterpoints = gpd.GeoDataFrame(
    districtswise_waterpoints, geometry='geometry')
districtswise_waterpoints.crs = CRS.from_epsg(4326)

In [ ]:
districtswise_waterpoints['geoid'] = districtswise_waterpoints.index.astype(str)

In [ ]:
m = folium.Map([-6.241,35.679], zoom_start=6)

choropleth = folium.Choropleth(
    geo_data=districtswise_waterpoints,
    data=districtswise_waterpoints,
    columns=['geoid','total_waterpoints'],
    key_on='feature.id',
    nan_fill_color='purple',
    nan_fill_opacity=0.4,
    fill_color='YlOrRd',
    highlight=True
).add_to(m)

choropleth.geojson.add_child(folium.features.GeoJsonTooltip(
        fields=['NAME_1','NAME_2','total_waterpoints'],
        aliases=['Region','District','Waterpoints'],
        style=('background-color: grey; color: white;'),
        localize=True
        )
)

m

In [ ]:
ward_url = 'https://raw.githubusercontent.com/thadk/GeoTZ/master/TZA_adm3.geojson'
wards_json_data = json.loads(requests.get(ward_url).text)
wards_df = gpd.GeoDataFrame.from_features(wards_json_data, crs='EPSG:4326')

In [ ]:
grouped_by_wards = train_df.groupby(['region','lga','ward'])

In [ ]:
wardswise_waterpoints = pd.DataFrame()
regions = []
districts = []
wards = []
waterpoints = []
for name,group in grouped_by_wards:
    if(name[0] == 'Dar es Salaam'):
        regions.append('Dar-Es-Salaam')
    else:
        regions.append(name[0])
    districts.append(name[1])
    wards.append(name[2])
    waterpoints.append(len(group))
wardswise_waterpoints['region'] = regions
wardswise_waterpoints['district'] = districts
wardswise_waterpoints['ward'] = wards
wardswise_waterpoints['total_waterpoints'] = waterpoints

In [ ]:
merged = wards_df.merge(wardswise_waterpoints,left_on=['NAME_1','NAME_2','NAME_3'],right_on=['region','district','ward'],how='left')

In [ ]:
wardswise_waterpoints = merged[['NAME_1','NAME_2','NAME_3','total_waterpoints','geometry']]

In [ ]:
wardswise_waterpoints = gpd.GeoDataFrame(
    wardswise_waterpoints, geometry='geometry')
wardswise_waterpoints.crs = CRS.from_epsg(4326)

In [ ]:
wardswise_waterpoints['geoid'] = wardswise_waterpoints.index.astype(str)

In [ ]:
m = folium.Map([-6.241,35.679], zoom_start=6)

choropleth = folium.Choropleth(
    geo_data=wardswise_waterpoints,
    data=wardswise_waterpoints,
    columns=['geoid','total_waterpoints'],
    key_on='feature.id',
    nan_fill_color='white',
    nan_fill_opacity=0.7,
    fill_color='OrRd',
    fill_opacity=0.9,
    line_opacity=0.2,
    line_color='white',
    highlight=False
).add_to(m)

m

WaterPoints Marker with population:

In [ ]:
basins_url = 'http://geoportal.icpac.net/geoserver/wfs?srsName=EPSG%3A4326&typename=geonode%3Atza_water_areas_dcw&outputFormat=json&version=1.0.0&service=WFS&request=GetFeature'
basins_json_data = json.loads(requests.get(basins_url).text)
basins_df = gpd.GeoDataFrame.from_features(basins_json_data, crs='EPSG:4326')

In [ ]:
m = folium.Map(
    location=[-6.241,35.679],
    tiles='Stamen Toner',
    zoom_start=6,min_zoom=3
)

marker_cluster = MarkerCluster(
    name='waterpoints clustered icons',
    overlay=True,
    control=False,
    icon_create_function=None
)
size = len(train_df)
for k in range(size):
    location = train_df['latitude'][k], train_df['longitude'][k]
    marker = folium.Marker(location=location)
    #popup = 'Region:{}<br>District:{}<br>Ward:{}<br>Sub-village:{}'.format(str(train_df['region'][k]),
                                                                           #str(train_df['lga'][k]),str(train_df['ward'][k]),
                                                                           #str(train_df['subvillage'][k]))
    #folium.Popup(popup).add_to(marker)
    marker_cluster.add_child(marker)

marker_cluster.add_to(m)

#folium.GeoJson(basins_df,name='basins_tanzania').add_to(m)

folium.LayerControl().add_to(m)


In [ ]:
#m.save(os.path.join('results', 'All_waterpoints.html'))

m

Waterpoints divided by the status:

In [ ]:
m = folium.Map(
    location=[-6.241,35.679],
    tiles='Stamen Toner',
    control_scale=True,zoom_start=6,
    min_zoom=3
)

size = len(train_df)

for k in range(size):
    location = train_df['latitude'][k], train_df['longitude'][k]
    status = train_df['status_group'][k]
    
    if(status == 'functional'):
        folium.Circle(
          location=location,
          radius=1000,
          color='#00ff00',
          fill=True,
          fill_color='#00ff00',
          #tooltip="<div><ul style='color: #444;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
        #"<li>Subvillage: "+str(train_df['subvillage'][k])+"</li>"+
        #"<li>Accessed by: "+str(train_df['population'])+"</li>"+
        #"</ul></div>"
       ).add_to(m)
    elif(status == 'non functional'):
        folium.Circle(
          location=location,
          radius=3000,
          color='#ff0000',
          fill=True,
          fill_color='#ff0000',
           # tooltip="<ul style='color: #444;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
        #"<li>Subvillage: "+str(train_df['subvillage'][k])+"</li>"+
        #"<li>Accessed by: "+str(train_df['population'])+"</li>"+
        #"</ul>"
       ).add_to(m)
    else:
        folium.Circle(
          location=location,
          radius=2000,
          color='#0000ff',
          fill=True,
          fill_color='#0000ff',
           # tooltip="<ul style='color: #444;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
        #"<li>Subvillage: "+str(train_df['subvillage'][k])+"</li>"+
        #"<li>Accessed by: "+str(train_df['population'])+"</li>"+
        #"</ul>"
       ).add_to(m)

In [ ]:
#m.save(os.path.join('results', 'waterpoints_by_status.html'))

m

In [ ]:
m = folium.Map(
    location=[-6.241,35.679],
    tiles='Stamen Toner',
    control_scale=True,zoom_start=6,
    min_zoom=3
)

size = len(train_df)

for k in range(size):
    location = train_df['latitude'][k], train_df['longitude'][k]
    quantity = train_df['quantity_group'][k]
    
    if(quantity == 'enough'):
        folium.Circle(
          location=location,
          radius=1000,
          color='#FFA07A',
          fill=True,
          fill_color='#FFA07A',
          #tooltip="<div><ul style='color: #444;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
        #"<li>Subvillage: "+str(train_df['subvillage'][k])+"</li>"+
        #"<li>Accessed by: "+str(train_df['population'])+"</li>"+
        #"</ul></div>"
       ).add_to(m)
    elif(quantity == 'seasonal'):
        folium.Circle(
          location=location,
          radius=2000,
          color='#CD5C5C',
          fill=True,
          fill_color='#CD5C5C',
       ).add_to(m)
    elif(quantity == 'dry'):
        folium.Circle(
          location=location,
          radius=4000,
          color='#FF0000',
          fill=True,
          fill_color='#FF0000',
       ).add_to(m)
    elif(quantity == 'insufficient'):
        folium.Circle(
          location=location,
          radius=5000,
          color='#8B0000',
          fill=True,
          fill_color='#8B0000',
       ).add_to(m)
    else:
        folium.Circle(
          location=location,
          radius=3000,
          color='#1E90FF',
          fill=True,
          fill_color='#1E90FF',
       ).add_to(m)

In [ ]:
#m.save(os.path.join('results', 'waterpoints_by_quantity.html'))

m

Quantity of water and status of the waterpoints are highly related.

In [ ]:
m = folium.Map(
    location=[-6.241,35.679],
    tiles='Stamen Toner',
    control_scale=True,zoom_start=6,
    min_zoom=3
)

size = len(train_df)

for k in range(size):
    location = train_df['latitude'][k], train_df['longitude'][k]
    quality = train_df['quality_group'][k]
    
    if(quality == 'good'):
        folium.Circle(
          location=location,
          radius=1000,
          color='#87CEFA',
          fill=True,
          fill_color='#87CEFA',
          #tooltip="<div><ul style='color: #444;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
        #"<li>Subvillage: "+str(train_df['subvillage'][k])+"</li>"+
        #"<li>Accessed by: "+str(train_df['population'])+"</li>"+
        #"</ul></div>"
       ).add_to(m)
    elif(quality == 'salty'):
        folium.Circle(
          location=location,
          radius=3000,
          color='#8A2BE2',
          fill=True,
          fill_color='#8A2BE2',
       ).add_to(m)
    elif(quality == 'milky'):
        folium.Circle(
          location=location,
          radius=4000,
          color='#4682B4',
          fill=True,
          fill_color='#4682B4',
       ).add_to(m)
    elif(quality == 'colored'):
        folium.Circle(
          location=location,
          radius=5000,
          color='#0000FF',
          fill=True,
          fill_color='#0000FF',
       ).add_to(m)
    elif(quality == 'fluoride'):
        folium.Circle(
          location=location,
          radius=6000,
          color='#00008B',
          fill=True,
          fill_color='#00008B',
       ).add_to(m)
    else:
        folium.Circle(
          location=location,
          radius=2000,
          color='#32CD32',
          fill=True,
          fill_color='#32CD32',
       ).add_to(m)

In [ ]:
#m.save(os.path.join('results', 'waterpoints_by_quality.html'))

m

m = folium.Map(
    location=[-6.241,35.679],
    tiles='Stamen Toner',
    control_scale=True,zoom_start=6,
    min_zoom=3
)

size = len(train_df)

for k in range(size):
    location = train_df['latitude'][k], train_df['longitude'][k]
    quantity = train_df['extraction_type'][k]
    
    if(quantity == 'enough'):
        folium.Circle(
          location=location,
          radius=1000,
          color='#FFA07A',
          fill=True,
          fill_color='#FFA07A',
          #tooltip="<div><ul style='color: #444;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
        #"<li>Subvillage: "+str(train_df['subvillage'][k])+"</li>"+
        #"<li>Accessed by: "+str(train_df['population'])+"</li>"+
        #"</ul></div>"
       ).add_to(m)
    elif(quantity == 'seasonal'):
        folium.Circle(
          location=location,
          radius=2000,
          color='#CD5C5C',
          fill=True,
          fill_color='#CD5C5C',
       ).add_to(m)
    elif(quantity == 'dry'):
        folium.Circle(
          location=location,
          radius=4000,
          color='#FF0000',
          fill=True,
          fill_color='#FF0000',
       ).add_to(m)
    elif(quantity == 'insufficient'):
        folium.Circle(
          location=location,
          radius=5000,
          color='#8B0000',
          fill=True,
          fill_color='#8B0000',
       ).add_to(m)
    else:
        folium.Circle(
          location=location,
          radius=3000,
          color='#1E90FF',
          fill=True,
          fill_color='#1E90FF',
       ).add_to(m)

In [ ]:
m = folium.Map(
    location=[-6.241,35.679],
    tiles='Stamen Toner',
    control_scale=True,zoom_start=6,
    min_zoom=3
)

size = len(train_df)

for k in range(size):
    location = train_df['latitude'][k], train_df['longitude'][k]
    quantity = train_df['quantity_group'][k]
    
    if(quantity == 'enough'):
        folium.Circle(
          location=location,
          radius=1000,
          color='#FFA07A',
          fill=True,
          fill_color='#FFA07A',
          #tooltip="<div><ul style='color: #444;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
        #"<li>Subvillage: "+str(train_df['subvillage'][k])+"</li>"+
        #"<li>Accessed by: "+str(train_df['population'])+"</li>"+
        #"</ul></div>"
       ).add_to(m)
    elif(quantity == 'seasonal'):
        folium.Circle(
          location=location,
          radius=2000,
          color='#CD5C5C',
          fill=True,
          fill_color='#CD5C5C',
       ).add_to(m)
    elif(quantity == 'dry'):
        folium.Circle(
          location=location,
          radius=4000,
          color='#FF0000',
          fill=True,
          fill_color='#FF0000',
       ).add_to(m)
    elif(quantity == 'insufficient'):
        folium.Circle(
          location=location,
          radius=5000,
          color='#8B0000',
          fill=True,
          fill_color='#8B0000',
       ).add_to(m)
    else:
        folium.Circle(
          location=location,
          radius=3000,
          color='#1E90FF',
          fill=True,
          fill_color='#1E90FF',
       ).add_to(m)

In [ ]:
m

--- A waterpoints longetivity could be taken as a new feature. Longetivity = construction year - date recorded(if not functional or needs repair)
--- Population can be divided into bins, same for amount_tsh and gps_height

In [ ]:
train_df.head()

In [ ]:

yearwise_df = train_df.groupby('construction_year')
year = []
waterpoints_number =  []
yearwise_waterpoints = pd.DataFrame()
for name,group in yearwise_df:
    year.append(name)
    waterpoints_number.append(len(group))
yearwise_waterpoints['year'] = year
yearwise_waterpoints['waterpoints_number'] = waterpoints_number
yearwise_waterpoints = yearwise_waterpoints[yearwise_waterpoints['year']>0]

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=yearwise_waterpoints['year'],y=yearwise_waterpoints['waterpoints_number'],
              mode='lines+markers',name="Waterpoints made every year"))
fig.show()

About 20,000 waterpoints have no recorded construction year. Out of the rest, maximum were made in 2010.

In [ ]:
fig = px.histogram(train_df,x='amount_tsh',nbins=20)
fig.show()

In [ ]:
fig = px.histogram(train_df, x="permit", color="status_group", title="Waterpoints status by their Permit ")
fig.show()

In [ ]:
fig = px.histogram(train_df, y="extraction_type", title="Extraction Type")
fig.show()

In [ ]:
status = train_df['status_group'].unique()

In [ ]:
for i in range(0,len(status)):
    col = 'extraction_type_group'
    stat = status[i]
    temp = train_df[train_df['status_group'] == stat]
    names = temp[col].unique()
    values = []
    for i in range(0,len(names)):
        values.append(len(temp[temp[col] == names[i]]))
    title = 'Extraction type in '+stat + ' waterpoints'
    fig = px.pie(values=values, names=names, title=title)
    fig.show()

In [ ]:
fig = px.histogram(train_df, x="payment", title="Waterpoints status by payment")
fig.show()

In [ ]:
for i in range(0,len(status)):
    col = 'payment_type'
    stat = status[i]
    temp = train_df[train_df['status_group'] == stat]
    names = temp[col].unique()
    values = []
    for i in range(0,len(names)):
        values.append(len(temp[temp[col] == names[i]]))
    title = 'Payment type in '+stat + ' waterpoints'
    fig = px.pie(values=values, names=names, title=title)
    fig.show()

In [ ]:
fig = px.histogram(train_df, x="water_quality",title="Status by Water Quality")
fig.show()

In [ ]:
for i in range(0,len(status)):
    col = 'quality_group'
    stat = status[i]
    temp = train_df[train_df['status_group'] == stat]
    names = temp[col].unique()
    values = []
    for i in range(0,len(names)):
        values.append(len(temp[temp[col] == names[i]]))
    title = 'Water quality type in '+stat + ' waterpoints'
    fig = px.pie(values=values, names=names, title=title)
    fig.show()

In [ ]:
fig = px.histogram(train_df, x="quantity",title="Status by Quantity")
fig.show()

In [ ]:
for i in range(0,len(status)):
    col = 'quantity_group'
    stat = status[i]
    temp = train_df[train_df['status_group'] == stat]
    names = temp[col].unique()
    values = []
    for i in range(0,len(names)):
        values.append(len(temp[temp[col] == names[i]]))
    title = 'Water quantity type in '+stat + ' waterpoints'
    fig = px.pie(values=values, names=names, title=title)
    fig.show()

In [ ]:
fig = px.histogram(train_df, x="source",title="Status by Source")
fig.show()

In [ ]:
for i in range(0,len(status)):
    col = 'source_type'
    stat = status[i]
    temp = train_df[train_df['status_group'] == stat]
    names = temp[col].unique()
    values = []
    for i in range(0,len(names)):
        values.append(len(temp[temp[col] == names[i]]))
    title = 'Water source type in '+stat + ' waterpoints'
    fig = px.pie(values=values, names=names, title=title)
    fig.show()

In [ ]:
for i in range(0,len(status)):
    col = 'source_class'
    stat = status[i]
    temp = train_df[train_df['status_group'] == stat]
    names = temp[col].unique()
    values = []
    for i in range(0,len(names)):
        values.append(len(temp[temp[col] == names[i]]))
    title = 'Water source class in '+stat + ' waterpoints'
    fig = px.pie(values=values, names=names, title=title)
    fig.show()

In [ ]:
fig = px.histogram(train_df, x="waterpoint_type",title="Status by Waterpoint Type")
fig.show()

In [ ]:
for i in range(0,len(status)):
    col = 'waterpoint_type_group'
    stat = status[i]
    temp = train_df[train_df['status_group'] == stat]
    names = temp[col].unique()
    values = []
    for i in range(0,len(names)):
        values.append(len(temp[temp[col] == names[i]]))
    title = 'Water source type in '+stat + ' waterpoints'
    fig = px.pie(values=values, names=names, title=title)
    fig.show()

In [ ]:
fig = px.histogram(train_df, x="management",title="Status by management")
fig.show()

In [ ]:
for i in range(0,len(status)):
    col = 'management_group'
    stat = status[i]
    temp = train_df[train_df['status_group'] == stat]
    names = temp[col].unique()
    values = []
    for i in range(0,len(names)):
        values.append(len(temp[temp[col] == names[i]]))
    title = 'Water management type in '+stat + ' waterpoints'
    fig = px.pie(values=values, names=names, title=title)
    fig.show()

In [ ]:
#same charts for quantity and quantity group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,
#management,management_group

In [ ]:
temp = train_df[['population','amount_tsh']].sort_values('population',ascending=True)

In [ ]:
values = []
names=[]

for i in range(0,len(status)):
    col = 'population'
    stat = status[i]
    temp = train_df[train_df['status_group'] == stat]
   
    names.append(stat)
    values.append(temp[col].sum())
    
title = 'Population served by different waterpoints'
fig = px.pie(values=values, names=names, title=title)
fig.show()

In [ ]:
for i in range(0,len(status)):
    col = 'region'
    stat = status[i]
    temp = train_df[train_df['status_group'] == stat]
    names = temp[col].unique()
    values = []
    for i in range(0,len(names)):
        values.append(len(temp[temp[col] == names[i]]))
    title = 'Region wise '+ stat + ' waterpoints'
    fig = px.pie(values=values, names=names, title=title)
    fig.show()

In [ ]:
train_df['basin'].value_counts()

In [ ]:
basins_url = 'http://geoportal.icpac.net/geoserver/wfs?srsName=EPSG%3A4326&typename=geonode%3Atza_water_areas_dcw&outputFormat=json&version=1.0.0&service=WFS&request=GetFeature'
basins_json_data = json.loads(requests.get(basins_url).text)
basins_df = gpd.GeoDataFrame.from_features(basins_json_data, crs='EPSG:4326')

In [ ]:
m = folium.Map(
    location=[-6.241,35.679],
    tiles='Cartodb Positron',
    control_scale=True,zoom_start=6,
    min_zoom=3,
    max_zoom=7)


folium.GeoJson(basins_df,
               name='basins_tanzania').add_to(m)

folium.LayerControl().add_to(m)

m

Handling categorical variables:

In [ ]:
train_df['funder'].value_counts()

In [ ]:
train_df['installer'].value_counts()

In [ ]:
len(train_df['wpt_name'].unique())

In [ ]:
train_df = train_df.loc[:, train_df.columns != 'funder']
train_df = train_df.loc[:, train_df.columns != 'installer']
train_df = train_df.loc[:, train_df.columns != 'wpt_name']

In [ ]:
test_df = test_df.loc[:, test_df.columns != 'funder']
test_df = test_df.loc[:, test_df.columns != 'installer']
test_df = test_df.loc[:, test_df.columns != 'wpt_name']

In [ ]:
train_df.shape

In [ ]:
train_df.head()

In [ ]:
train_df['basin'].unique()

In [ ]:
label_encoder = preprocessing.LabelEncoder()

In [ ]:
train_df['basin']= label_encoder.fit_transform(train_df['basin']) 

In [ ]:
test_df['basin']= label_encoder.fit_transform(test_df['basin']) 

In [ ]:
len(train_df['region'].unique()) == len(train_df['region_code'].unique())

In [ ]:
train_df['region'].value_counts()

In [ ]:
train_df['region_code'].unique()

In [ ]:
len(train_df['district_code'].unique()) == len(train_df['lga'].unique())

In [ ]:
train_df['district_code'].unique()

In [ ]:
train_df[['lga','district_code','region_code','region']]

In [ ]:
train_df['lga']= label_encoder.fit_transform(train_df['lga']) 

In [ ]:
test_df['lga']= label_encoder.fit_transform(test_df['lga']) 

In [ ]:
train_df[['region_code','region']].value_counts()

In [ ]:
train_df['region']= label_encoder.fit_transform(train_df['region']) 

In [ ]:
test_df['region']= label_encoder.fit_transform(test_df['region']) 

In [ ]:
train_df['ward'].value_counts()

In [ ]:
temp = train_df['ward'].value_counts()
ward_data = pd.DataFrame()
ward_data['ward'] = temp.index
ward_data['count'] = temp.values

In [ ]:
other_wards = ward_data.loc[ward_data['count'] <= 10,'ward']

In [ ]:
other_wards.reset_index(drop=True)

In [ ]:
train_df1 = train_df.copy()
other_wards = other_wards.to_numpy()
def group_other_wards(ward):
    if ward in other_wards:
        return 'other'
    else:
        return ward
    
train_df1['ward'] = train_df['ward'].apply(lambda x:group_other_wards(x),1)

In [ ]:
temp = test_df['ward'].value_counts()
ward_data1 = pd.DataFrame()
ward_data1['ward'] = temp.index
ward_data1['count'] = temp.values

other_wards1 = ward_data1.loc[ward_data1['count'] <= 10,'ward']

other_wards1.reset_index(drop=True)

test_df1 = test_df.copy()
other_wards1 = other_wards1.to_numpy()
def group_other_wards(ward):
    if ward in other_wards1:
        return 'other'
    else:
        return ward
    
test_df1['ward'] = test_df['ward'].apply(lambda x:group_other_wards(x),1)

In [ ]:
train_df1['ward'].value_counts()

In [ ]:
train_df['subvillage'].value_counts()

In [ ]:
temp = train_df['subvillage'].value_counts()
village_data = pd.DataFrame()
village_data['subvillage'] = temp.index
village_data['count'] = temp.values

In [ ]:
village_data

In [ ]:
village_data[village_data['count'] > 10]

In [ ]:

other_villages = village_data.loc[village_data['count'] <= 10,'subvillage']
other_villages.reset_index(drop=True)

other_villages = other_villages.to_numpy()
def group_other_villages(village):
    if village in other_villages:
        return 'other'
    else:
        return village
    
train_df1['subvillage'] = train_df1['subvillage'].apply(lambda x:group_other_villages(x),1)

In [ ]:
train_df1['subvillage'].value_counts()

In [ ]:
temp = test_df['subvillage'].value_counts()
village_data1 = pd.DataFrame()
village_data1['subvillage'] = temp.index
village_data1['count'] = temp.values


other_villages1 = village_data1.loc[village_data1['count'] <= 10,'subvillage']
other_villages1.reset_index(drop=True)

other_villages1 = other_villages1.to_numpy()
def group_other_villages(village):
    if village in other_villages1:
        return 'other'
    else:
        return village
    
test_df1['subvillage'] = test_df1['subvillage'].apply(lambda x:group_other_villages(x),1)


In [ ]:
train_df1['ward']= label_encoder.fit_transform(train_df1['ward']) 
train_df1['subvillage']= label_encoder.fit_transform(train_df1['subvillage']) 

In [ ]:
test_df1['ward']= label_encoder.fit_transform(test_df1['ward']) 
test_df1['subvillage']= label_encoder.fit_transform(test_df1['subvillage']) 

In [ ]:
train_df1.head()

In [ ]:
test_df.head()

In [ ]:
train_df1['public_meeting'].unique()

In [ ]:
train_df1['permit'].unique()

In [ ]:

def convert_to_bool(val):
    if val == 'Unknown':
        return -1
    elif val == False:
        return 0
    else:
        return 1
    
train_df1['public_meeting'] = train_df1['public_meeting'].apply(convert_to_bool)
train_df1['permit'] = train_df1['permit'].apply(convert_to_bool)

In [ ]:
test_df1['public_meeting'] = test_df1['public_meeting'].apply(convert_to_bool)
test_df1['permit'] = test_df1['permit'].apply(convert_to_bool)

In [ ]:
train_df1.head()

In [ ]:
train_df1['scheme_management'].unique()

In [ ]:
train_df1['extraction_type']= label_encoder.fit_transform(train_df1['extraction_type']) 
train_df1['extraction_type_group']= label_encoder.fit_transform(train_df1['extraction_type_group']) 
train_df1['extraction_type_class']= label_encoder.fit_transform(train_df1['extraction_type_class']) 
train_df1['scheme_management']= label_encoder.fit_transform(train_df1['scheme_management']) 
train_df1['management']= label_encoder.fit_transform(train_df1['management']) 
train_df1['management_group']= label_encoder.fit_transform(train_df1['management_group']) 
train_df1['payment']= label_encoder.fit_transform(train_df1['payment']) 
train_df1['payment_type']= label_encoder.fit_transform(train_df1['payment_type']) 
train_df1['water_quality']= label_encoder.fit_transform(train_df1['water_quality']) 
train_df1['quality_group']= label_encoder.fit_transform(train_df1['quality_group']) 
train_df1['quantity']= label_encoder.fit_transform(train_df1['quantity']) 
train_df1['quantity_group']= label_encoder.fit_transform(train_df1['quantity_group']) 
train_df1['source']= label_encoder.fit_transform(train_df1['source']) 
train_df1['source_type']= label_encoder.fit_transform(train_df1['source_type']) 
train_df1['source_class']= label_encoder.fit_transform(train_df1['source_class']) 
train_df1['waterpoint_type']= label_encoder.fit_transform(train_df1['waterpoint_type']) 
train_df1['waterpoint_type_group']= label_encoder.fit_transform(train_df1['waterpoint_type_group']) 

In [ ]:
train_df1.head()

In [ ]:
test_df1['extraction_type']= label_encoder.fit_transform(test_df1['extraction_type']) 
test_df1['extraction_type_group']= label_encoder.fit_transform(test_df1['extraction_type_group']) 
test_df1['extraction_type_class']= label_encoder.fit_transform(test_df1['extraction_type_class']) 
test_df1['scheme_management']= label_encoder.fit_transform(test_df1['scheme_management']) 
test_df1['management']= label_encoder.fit_transform(test_df1['management']) 
test_df1['management_group']= label_encoder.fit_transform(test_df1['management_group']) 
test_df1['payment']= label_encoder.fit_transform(test_df1['payment']) 
test_df1['payment_type']= label_encoder.fit_transform(test_df1['payment_type']) 
test_df1['water_quality']= label_encoder.fit_transform(test_df1['water_quality']) 
test_df1['quality_group']= label_encoder.fit_transform(test_df1['quality_group']) 
test_df1['quantity']= label_encoder.fit_transform(test_df1['quantity']) 
test_df1['quantity_group']= label_encoder.fit_transform(test_df1['quantity_group']) 
test_df1['source']= label_encoder.fit_transform(test_df1['source']) 
test_df1['source_type']= label_encoder.fit_transform(test_df1['source_type']) 
test_df1['source_class']= label_encoder.fit_transform(test_df1['source_class']) 
test_df1['waterpoint_type']= label_encoder.fit_transform(test_df1['waterpoint_type']) 
test_df1['waterpoint_type_group']= label_encoder.fit_transform(test_df1['waterpoint_type_group']) 

In [ ]:
train_df1.head()

In [ ]:

def longetivity_feature(row):
    if (row['construction_year'] == 0):
        return -1
    else:
        val = row['date_recorded'].year - row['construction_year']
        return val
train_df1['longetivity'] = train_df1.apply(lambda row:longetivity_feature(row),1)

In [ ]:
test_df1['longetivity'] = test_df1.apply(lambda row:longetivity_feature(row),1)

Later, we can calculate bins for amount_tsh and gps_height. For now, only normalize them.

In [ ]:
train_df1['amount_tsh'].describe()

In [ ]:
bins = range(0,350001,50)
labels = range(1,7001)
train_df1['amount_tsh'] = train_df1['amount_tsh'].astype(int)
train_df1['amount_tsh_binned'] = pd.cut(train_df1['amount_tsh'], bins=bins, labels=labels)
test_df1['amount_tsh'] = test_df1['amount_tsh'].astype(int)
test_df1['amount_tsh_binned'] = pd.cut(test_df1['amount_tsh'], bins=bins, labels=labels)

In [ ]:
train_df1['amount_tsh_binned'] = train_df1['amount_tsh_binned'].astype(float)
train_df1['amount_tsh_binned'] = train_df1['amount_tsh_binned'].fillna(0)
train_df1['amount_tsh_binned'] = train_df1['amount_tsh_binned'].astype(int)

In [ ]:
test_df1['amount_tsh_binned'] = test_df1['amount_tsh_binned'].astype(float)
test_df1['amount_tsh_binned'] = test_df1['amount_tsh_binned'].fillna(0)
test_df1['amount_tsh_binned'] = test_df1['amount_tsh_binned'].astype(int)

In [ ]:
train_df1['gps_height'].describe()

In [ ]:
bins = range(-91,2790,30)
labels = range(1,97)
train_df1['gps_height'] = train_df1['gps_height'].astype(int)
train_df1['gps_height_binned'] = pd.cut(train_df1['gps_height'], bins=bins, labels=labels)
test_df1['gps_height'] = test_df1['gps_height'].astype(int)
test_df1['gps_height_binned'] = pd.cut(test_df1['gps_height'], bins=bins, labels=labels)

In [ ]:
train_df1.head()

In [ ]:
train_df1[train_df1['gps_height'] == -90]

In [ ]:
train_df1['gps_height_binned'] = train_df1['gps_height_binned'].astype(int)

In [ ]:
test_df1['gps_height_binned'] = test_df1['gps_height_binned'].astype(int)

Handling Geospatial Data:

There should be **9** clusters

In [ ]:
kms_per_radian = 6371.0088

In [ ]:
def dbscan_reduce(df,epsilon):
    coords = df[['latitude', 'longitude']].values
    dbscan = DBSCAN(eps=epsilon,min_samples=1,algorithm='ball_tree',metric='haversine').fit(np.radians(coords))
    labels = dbscan.labels_
    n_clusters = len(set(labels))
    print('number of clusters:',n_clusters)
    clusters = [coords[labels == n] for  n in range(n_clusters)]
    return clusters

In [ ]:
epsilon = 5 / kms_per_radian
clusters = dbscan_reduce(train_df1,epsilon)

In [ ]:
epsilon = 5 / kms_per_radian
clusters_test = dbscan_reduce(test_df1,epsilon)

In [ ]:
clusters

In [ ]:
def get_clusters(data):
    for i in range(0,len(clusters)):
        for j in range(0,len(clusters[i])):
            data.loc[(data['latitude'] == clusters[i][j][0]) & (data['longitude'] == clusters[i][j][1]),'location_cluster'] = i

In [ ]:
get_clusters(train_df1)

In [ ]:
get_clusters(test_df1)

In [ ]:
train_df1.head(20)

In [ ]:
columns = ['amount_tsh_binned','gps_height_binned','basin','subvillage','region','lga','ward','population','public_meeting',
           'scheme_management','permit','extraction_type','extraction_type_group','extraction_type_class','management',
           'management_group','payment','payment_type','water_quality','quality_group','quantity','quantity_group',
           'source','source_type','source_class','waterpoint_type','waterpoint_type_group','longetivity','location_cluster']
X = train_df1[columns]
y = train_df1['status_group']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=27)

In [ ]:
test_data = test_df1[columns]

In [ ]:
scaled = MinMaxScaler().fit(X_train)

In [ ]:
X_train1 = scaled.transform(X_train)
X_test1 = scaled.transform(X_test)

In [ ]:
X_train_scaled = pd.DataFrame(X_train1, index=X_train.index, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test1, index=X_test.index, columns=X_test.columns)

In [ ]:
y_train= label_encoder.fit_transform(y_train) 
y_test= label_encoder.fit_transform(y_test) 

In [ ]:
scaled_test = MinMaxScaler().fit(test_data)
test_data1 = scaled.transform(test_data)
test_scaled = pd.DataFrame(test_data1, index=test_data.index, columns=test_data.columns)

non-functional -2, functional - 0, needs repair- 1

In [ ]:
def classification_rate(y_pred):
    count = 0
    for i in range(0,len(y_pred)):
        if y_pred[i] == y_test[i]:
            count = count + 1
    rate = count / len(y_test)
    return rate

KNN:

In [ ]:
leaf_size = list(range(3,15))
n_neighbors = list(range(1,12))

hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors)

In [ ]:
model = KNeighborsClassifier()
model_knn = GridSearchCV(model, hyperparameters, cv=10,n_jobs=-1)
best_model_knn = model_knn.fit(X_train,y_train)

print('Best leaf_size:', best_model_knn.best_estimator_.get_params()['leaf_size'])
#print('Best p:', best_model_knn.best_estimator_.get_params()['p'])
print('Best n_neighbors:', best_model_knn.best_estimator_.get_params()['n_neighbors'])

In [ ]:
model_knn1 = KNeighborsClassifier(leaf_size=5,n_neighbors = 3,p=1)
model_knn1.fit(X_train,y_train)
y_pred = model_knn1.predict(X_test)
accuracy_knn = accuracy_score(y_test, y_pred)
print('Classification rate',classification_rate(y_pred))
print('Accuracy:',accuracy_knn)
print(classification_report(y_test, y_pred))

XGBClassifier

In [ ]:
xgb_clf = xgb.XGBClassifier(verbosity=3,objective='multi:softmax')
parameters = {"learning_rate": [0.1, 0.01, 0.001],
               "gamma" : [0.01, 0.1, 0.3, 0.5, 1, 1.5, 2],
               "max_depth": [2, 4, 7, 10],
               "colsample_bytree": [0.3, 0.6, 0.8, 1.0],
               "subsample": [0.2, 0.4, 0.5, 0.6, 0.7],
               "n_estimators": [250, 500, 1000, 2000, 3000]}
xgb_rscv = RandomizedSearchCV(xgb_clf,param_distributions=parameters,cv = 7, verbose = 3, random_state = 40,scoring='f1_micro')
model_xgboost = xgb_rscv.fit(X_train, y_train)
params = model_xgboost.best_estimator_.get_params()

In [ ]:
model_xgb = xgb.XGBClassifier(colsample_bylevel= 1,colsample_bynode= 1, colsample_bytree= 0.8, gamma= 1,
                              learning_rate= 0.1,max_depth= 4,n_estimators= 3000,subsample= 0.7,objective='multi:softprob')
model_xgb.fit(X_train,y_train)
y_pred_xgb = model_xgb.predict(X_test)
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print('Classification rate',classification_rate(y_pred_xgb))
print('Accuracy score:',accuracy_xgb)
print(classification_report(y_test, y_pred_xgb))

In [ ]:
pred_test = model_xgb.predict(test_df[columns])
test_pred_df = pd.DataFrame()
test_pred_df['id'] = test_df1['id']
test_pred_df['status_group'] = pred_test

In [ ]:
fig = plot_importance(model_xgb)
plt.figure(figsize=(10,8))
plt.show()

In [ ]:


cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)

Random Forrest:

In [ ]:
# rf = RandomForestRegressor(random_state = 42)
# params_rg = {'bootstrap': [True, False],
#  'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
#  'max_features': ['auto', 'sqrt'],
#  'min_samples_leaf': [1, 2, 4],
#  'min_samples_split': [2, 5, 10],
#  'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}
# xgb_rscv = RandomizedSearchCV(xgb_clf,param_distributions=parameters,cv = 7, random_state = 40,scoring='f1_micro',n_jobs=-1)
# model_xgboost = xgb_rscv.fit(X_train, y_train)
# params = model_xgboost.best_estimator_.get_params()

In [ ]:
model_rf = RandomForestClassifier(random_state = 1, max_depth = 15, n_estimators = 2000, min_samples_split = 2, min_samples_leaf = 1)
                                   
model_rf.fit(X_train, y_train)
y_pred_rf = model_rf.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print('Classification rate',classification_rate(y_pred_rf))
print('Accuracy score:',accuracy_rf)
print(classification_report(y_test, y_pred_rf))